In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as W
from selenium.webdriver.support import expected_conditions as E
from selenium.webdriver import ActionChains
import time
import pandas as pd

# Helper Function to manipulate raw data

In [2]:
#check duplication of restaurant 
def duplicate(name_list, name):
    if name in name_list:
        return True
    else: 
        name_list.append(name)
        return False

In [3]:
# Getting number from a string
def get_num(txt):
    return [s for s in txt if s.isdigit()]

In [4]:
#prepare panda data
mainframe = pd.DataFrame({'name':[],'price':[],'bookmark':[],'happy':[],'ok':[],'sad':[],
                        'food_type':[],'location':[],'branch':[], 'number_seat':[], 'payment':[],
                        'overall':[],'taste':[],'decor':[],'service':[],'hygiene':[],'value':[]})
def panda_data(oneres):
    addrow = pd.DataFrame([oneres],columns=['name','price','bookmark','happy','ok','sad','food_type','location',
                                            'branch', 'number_seat', 'payment',
                                            'overall','taste','decor','service','hygiene','value']) #adding additional row
    global mainframe
    mainframe=mainframe.append(addrow,ignore_index = True)

# Helper Function for getting data

In [5]:
# get more info by clicking into the individual restaurant
def get_more_info(bs_more):
    time.sleep(3)

    #get rating
    try:
        rate_list = bs_more.find_all ('div',{'class':'header-score-details-right-item'})
        overall = bs_more.find('div',class_='header-score').get_text()
        criteria = {'overall': overall}
        for rate_item in rate_list:
            c_name = str(rate_item.find('div',{'class':'header-score-details-right-item-title'}).get_text()) 
            rate = rate_item.find_all('div')[1].attrs['class'][2]
            rate = str(get_num(rate)[0])
            criteria[c_name]=rate
    except:  return None
        
    # get number of branches
    try:
        num_branch=bs_more.find('section',class_='all-branches-section').a.get_text()
        num_branch = get_num(num_branch)
    except:
        num_branch = 1

    rest_info = bs_more.find('div',class_='left-col').find('div',class_='or-section-group pois-filter-feature')
    # get payment method
    pay_list = []
    try:
        payment = rest_info.find('div',class_="comma-tags").find_all('span')
        for pay in payment:
            pay_list.append(str(pay.get_text()))
    except: pass

    #get num_seat
    try:
        num_seat = str(rest_info.find('div',class_='content'))
        num_seat=str(''.join(get_num(num_seat)))
    except:  return None

    # other facility
    try:
        rest_info = bs_more.find('div',class_='left-col').find('div',class_='or-section-group pois-filter-feature')
        other_info = rest_info.find_all('span',class_='condition-name')
        other_facility = []
        for item in other_info:
            other_facility+=item.contents
    except: return None

    #satisfaction level
    try:
        satisfaction_list = bs_more.find_all('div',class_='score-div')
        satisfaction = {}
        score_cate = ['happy','ok','sad']
        for i in range(3):
            satisfaction[score_cate[i]] = satisfaction_list[i].get_text()
    except: return None

    return list(criteria.values()), num_branch, pay_list, num_seat, other_facility, satisfaction

In [6]:
#get info for each district
def get_info(bs, district):
    global name_list
    item_list =  bs.find_all('section',class_='content-wrapper')

    #get info of each restaurant
    for item in item_list:
        name = item.find('h2').a.string
        
        #enter new page for extra info
        try:
            moreinfo = W(driver, 5).until(E.presence_of_element_located((By.LINK_TEXT,str(name))))
            moreinfo.click()
            time.sleep(5)
            bs_more=BeautifulSoup(driver.page_source, 'html.parser')
            bs_more.prettify()
            try:
                criteria, num_branch, pay_list, num_seat, other_facility, satisfaction = get_more_info(bs_more)
            except: continue
            print('enter',name)

            try:
                driver.execute_script("window.history.go(-1)")
                print('go_out')
            except:
                print('go back failed')
                driver.quit()
        except:
            print('cannot',name); continue
        
        #go back main page get info 
        try:
            price = item.find('div',{'class':'icon-info icon-info-food-price'}).span.string
        except AttributeError:
            continue
        
        try:
            bookmark = item.find('div',{'class':'text bookmarkedUserCount js-bookmark-count'}).get("data-count")
        except AttributeError:
            continue
            
        try:
            food_type = item.find('li').string
        except AttributeError:
            continue

        try:
            location = item.find('div',{'class':'icon-info address'}).a.string
            if location != district: continue
        except AttributeError:
            continue

        #check duplication and add to the pandaframe
        if not duplicate(name_list,name):
            one_res = [name, price, bookmark,
                        satisfaction['happy'], satisfaction['ok'], satisfaction['sad'],
                        food_type,location, num_branch, num_seat,pay_list]
            for i in criteria:
                one_res.append(i)
            panda_data(one_res)

# Main function using selenium

In [7]:
#prepare url for searching other district
url_main = 'https://www.openrice.com'
url_district = ['Causeway Bay','Mong Kok','Central','Tsim Sha Tsui','Yuen Long','Tsuen Wan']

In [8]:
#start chrome
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.openrice.com/en/hongkong/restaurants")

In [ ]:
#main
cookie_pressed = False
name_list = []
for district in url_district:
    #find search box
    try:
        search_box = driver.find_element_by_name("where")
        search_box.clear() 
        search_box.send_keys(district)
        search = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[4]/div/button').click()
        time.sleep(2)
    except:
        print('error'); driver.quit()

    #ready to get info
    endsearch = False; page = 1; info_list = []
    
    while not endsearch:
        #cookies
        try:
            if (page == 1 and not cookie_pressed ):
                print('cookie pressing')
                cookie = W(driver, 2).until(E.presence_of_element_located((By.XPATH,'//*[@id="cookies-agreement"]/div/button')))
                type(cookie)
                cookie.click()
                cookie_pressed = True
                #driver.implicitly_wait(5)
        except:
            print("no cookies")

        bs = BeautifulSoup(driver.page_source, 'html.parser')
        bs.prettify()
        time.sleep(2)
        get_info(bs,district)
       
        #go to next page
        # find the link of next page 
        try:
            nextpage = W(driver, 5).until(E.presence_of_element_located((By.LINK_TEXT,str(page))))
            #actions = ActionChains(driver)
            #actions.move_to_element(nextpage)
        except:
            endsearch = True
            break
       #click on next page
        if not nextpage:
            endsearch = True
        else:
            print(page)
            page +=1
            nextpage.click()
            #actions.click(on_element=nextpage).perform()
            time.sleep(5)

    print('done',district)

driver.quit()

#dataframe to csv

mainframe.to_csv('restaurant_info_6Districts.csv',sheet_name='restaurant info')
        

cookie pressing
enter Mamaday
go_out
cannot Dodam Chicken (利舞臺廣場)
cannot Little Vegas
cannot Sodam Chicken
cannot The Grill Room (The L. Square)
cannot Kobekyu (Bartlock Centre) 吉列牛忌廉烏冬專門店 (百樂中心)
cannot Tsukiji Japanese Restaurant (Island Beverley) 築地日本料理 (金百利廣場)
cannot Bingo & Cook
cannot Toretore Hamayaki 漁獲浜燒
